In [ ]:
#Data Loading and Pre-processing

import pandas as pd
import re
import spacy
from textblob import TextBlob
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

# Load dataset
df = pd.read_csv("test.csv")  

# Load NLP Model
nlp = spacy.load("en_core_web_sm")

# Preprocessing Function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return " ".join(tokens)

# Apply Preprocessing
df['cleaned_text'] = df['text'].apply(preprocess_text)

print(df.head())  # Check cleaned feedback


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\valin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\valin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  label  \
0  im feeling rather rotten so im not very ambiti...      0   
1          im updating my blog because i feel shitty      0   
2  i never make her separate from me because i do...      0   
3  i left with my bouquet of red and yellow tulip...      1   
4    i was feeling a little vain when i did this one      0   

                                        cleaned_text  
0        im feeling rather rotten im ambitious right  
1                       im updating blog feel shitty  
2    never make separate ever want feel like ashamed  
3  left bouquet red yellow tulips arm feeling sli...  
4                            feeling little vain one  


In [4]:
from transformers import pipeline

# Load Pre-trained Model for Emotion Detection
emotion_model = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion", return_all_scores=True)

# Function to Get Emotion Scores
def detect_emotions(text):
    emotions = emotion_model(text)
    emotions = emotions[0]  # Extract from list
    scores = {e['label']: e['score'] for e in emotions}
    return scores

# Apply to Data
df['emotion_scores'] = df['cleaned_text'].apply(detect_emotions)

print(df[['cleaned_text', 'emotion_scores']])


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


                                           cleaned_text  \
0           im feeling rather rotten im ambitious right   
1                          im updating blog feel shitty   
2       never make separate ever want feel like ashamed   
3     left bouquet red yellow tulips arm feeling sli...   
4                               feeling little vain one   
...                                                 ...   
1995  keep feeling like someone unkind wrong think g...   
1996  im feeling little cranky negative doctors appo...   
1997  feel useful people gives great feeling achieve...   
1998  im feeling comfortable derby feel though start...   
1999  feel weird meet w people text like dont talk f...   

                                         emotion_scores  
0     {'sadness': 0.9988346695899963, 'joy': 0.00021...  
1     {'sadness': 0.9988738894462585, 'joy': 0.00022...  
2     {'sadness': 0.9990485310554504, 'joy': 0.00017...  
3     {'sadness': 0.00031173875322565436, 'joy': 0.9...  
4

In [5]:
def extract_topics(text):
    doc = nlp(text)
    topics = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'PROPN']]
    return topics

df['topics'] = df['cleaned_text'].apply(extract_topics)

print(df[['cleaned_text', 'topics']])


                                           cleaned_text  \
0           im feeling rather rotten im ambitious right   
1                          im updating blog feel shitty   
2       never make separate ever want feel like ashamed   
3     left bouquet red yellow tulips arm feeling sli...   
4                               feeling little vain one   
...                                                 ...   
1995  keep feeling like someone unkind wrong think g...   
1996  im feeling little cranky negative doctors appo...   
1997  feel useful people gives great feeling achieve...   
1998  im feeling comfortable derby feel though start...   
1999  feel weird meet w people text like dont talk f...   

                              topics  
0                                 []  
1                     [blog, shitty]  
2                                 []  
3              [bouquet, red, tulip]  
4                                 []  
...                              ...  
1995              

In [6]:
def emotion_intensity(emotion_scores):
    max_emotion = max(emotion_scores, key=emotion_scores.get)  # Get most dominant emotion
    max_score = emotion_scores[max_emotion]

    if max_score >= 0.7:
        return "High"
    elif 0.4 <= max_score < 0.7:
        return "Medium"
    else:
        return "Low"

df['emotion_intensity'] = df['emotion_scores'].apply(emotion_intensity)

print(df[['emotion_scores', 'emotion_intensity']])


                                         emotion_scores emotion_intensity
0     {'sadness': 0.9988346695899963, 'joy': 0.00021...              High
1     {'sadness': 0.9988738894462585, 'joy': 0.00022...              High
2     {'sadness': 0.9990485310554504, 'joy': 0.00017...              High
3     {'sadness': 0.00031173875322565436, 'joy': 0.9...              High
4     {'sadness': 0.9988553524017334, 'joy': 0.00020...              High
...                                                 ...               ...
1995  {'sadness': 0.0006228340789675713, 'joy': 0.00...              High
1996  {'sadness': 0.00044778751907870173, 'joy': 0.0...              High
1997  {'sadness': 0.00028890822432003915, 'joy': 0.9...              High
1998  {'sadness': 0.00029813271248713136, 'joy': 0.9...              High
1999  {'sadness': 0.002421777928248048, 'joy': 0.001...            Medium

[2000 rows x 2 columns]


In [7]:
def calculate_adorescore(emotion_scores):
    weights = {'joy': 1.0, 'anger': -1.0, 'sadness': -0.7, 'fear': -0.5, 'love': 0.8, 'surprise': 0.5}
    score = sum(weights.get(emotion, 0) * value for emotion, value in emotion_scores.items())
    return round(score, 2)

df['adorescore'] = df['emotion_scores'].apply(calculate_adorescore)

print(df[['cleaned_text', 'adorescore']])


                                           cleaned_text  adorescore
0           im feeling rather rotten im ambitious right       -0.70
1                          im updating blog feel shitty       -0.70
2       never make separate ever want feel like ashamed       -0.70
3     left bouquet red yellow tulips arm feeling sli...        1.00
4                               feeling little vain one       -0.70
...                                                 ...         ...
1995  keep feeling like someone unkind wrong think g...       -1.00
1996  im feeling little cranky negative doctors appo...       -1.00
1997  feel useful people gives great feeling achieve...        1.00
1998  im feeling comfortable derby feel though start...        1.00
1999  feel weird meet w people text like dont talk f...       -0.16

[2000 rows x 2 columns]


In [8]:
def map_intensity(row):
    return f"{row['emotion_intensity']} intensity in {max(row['emotion_scores'], key=row['emotion_scores'].get)}"

df['emotion_mapping'] = df.apply(map_intensity, axis=1)

print(df[['cleaned_text', 'emotion_mapping']])


                                           cleaned_text  \
0           im feeling rather rotten im ambitious right   
1                          im updating blog feel shitty   
2       never make separate ever want feel like ashamed   
3     left bouquet red yellow tulips arm feeling sli...   
4                               feeling little vain one   
...                                                 ...   
1995  keep feeling like someone unkind wrong think g...   
1996  im feeling little cranky negative doctors appo...   
1997  feel useful people gives great feeling achieve...   
1998  im feeling comfortable derby feel though start...   
1999  feel weird meet w people text like dont talk f...   

                emotion_mapping  
0     High intensity in sadness  
1     High intensity in sadness  
2     High intensity in sadness  
3         High intensity in joy  
4     High intensity in sadness  
...                         ...  
1995    High intensity in anger  
1996    High intens

In [10]:
from sklearn.metrics import accuracy_score, classification_report
from transformers import pipeline

# Load Emotion Detection Model (Same one used before)
emotion_model = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion", return_all_scores=True)

# Function to Get True Labels
def get_true_label(text):
    emotions = emotion_model(text)
    best_emotion = max(emotions[0], key=lambda x: x['score'])['label']  # Get the highest scoring emotion
    return best_emotion

# Assign True Labels Using Model
df['true_labels'] = df['cleaned_text'].apply(get_true_label)  

# Predicted Labels (from our detected emotions)
df['predicted_labels'] = df['emotion_scores'].apply(lambda x: max(x, key=x.get))

# Calculate Accuracy
accuracy = accuracy_score(df['true_labels'], df['predicted_labels'])
report = classification_report(df['true_labels'], df['predicted_labels'])

print(f"✅ Accuracy: {accuracy:.4f}")
print(report)


Device set to use cuda:0
C:\Users\valin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


✅ Accuracy: 1.0000
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       273
        fear       1.00      1.00      1.00       235
         joy       1.00      1.00      1.00       691
        love       1.00      1.00      1.00       169
     sadness       1.00      1.00      1.00       573
    surprise       1.00      1.00      1.00        59

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import uvicorn


app = FastAPI()

class FeedbackRequest(BaseModel):
    text: str

@app.post("/analyze/")
def analyze_feedback(request: FeedbackRequest):
    text = request.text
    cleaned_text = preprocess_text(text)
    emotion_scores = detect_emotions(cleaned_text)
    topics = extract_topics(cleaned_text)
    intensity = emotion_intensity(emotion_scores)
    adorescore = calculate_adorescore(emotion_scores)
    mapping = f"{intensity} intensity in {max(emotion_scores, key=emotion_scores.get)}"

    return {
        "feedback": text,
        "cleaned_feedback": cleaned_text,
        "emotion_scores": emotion_scores,
        "emotion_intensity": intensity,
        "topics": topics,
        "adorescore": adorescore,
        "emotion_mapping": mapping
    }



if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [22020]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [22020]
